### **Table 1: Absolute Number of Substance use + Heart Disease related Deaths Stratified by Age, Sex and Race/Ethnicity in the United States, 2005-2015**

In [8]:
import pandas as pd

# Load the data with low_memory=False to handle mixed types
mortality_df = pd.read_csv('SU_HD_mortality_final.csv', low_memory=False)
population_df = pd.read_csv('population_data_v1.csv', low_memory=False)

# Create mappings for age groups and race
age_group_map = {
    1: '25-39 years',
    2: '40-54 years', 
    3: '55-69 years',
    4: '70-84 years',
    5: '85+ years'
}
race_map = {
    1: 'White',
    2: 'Black/AA',
    3: 'AI/AN', 
    4: 'A/PI'
}

# Apply mappings to mortality data
mortality_df['age_group'] = mortality_df['age_recode_custom'].map(age_group_map)
mortality_df['race'] = mortality_df['race_recode_5'].map(race_map)

# Filter population data to only ages ≥25 (using age_recode_custom 1-5)
pop_25plus = population_df[population_df['age_recode_custom'].between(1,5)]
yearly_population = pop_25plus.groupby('Year')['Population'].sum().rename_axis('current_data_year')

def create_table_s2():
    # Group by year and count total deaths
    result = pd.DataFrame({'Overall': mortality_df.groupby('current_data_year').size()})
    
    # Add demographic breakdowns
    demographic_columns = [
        ('sex', {'F': 'Women', 'M': 'Men'}),
        ('race', None),
        ('age_group', None)
    ]
    
    for col, rename_dict in demographic_columns:
        counts = mortality_df.groupby(['current_data_year', col]).size().unstack(fill_value=0)
        if rename_dict:
            counts = counts.rename(columns=rename_dict)
        result = result.join(counts)
    
    # Add filtered population data
    result = result.join(yearly_population)
    
    # Create total row (summing across years)
    total_row = pd.DataFrame(result.sum()).T
    total_row.index = ['Total']
    
    # Combine annual data with total row
    final_table = pd.concat([result.reset_index(), total_row], ignore_index=True)
    final_table = final_table.rename(columns={'current_data_year': 'Year'})
    final_table['Year'] = final_table['Year'].fillna(0).astype(int)
    final_table['Year'] = final_table['Year'].astype(object)
    final_table.loc[final_table.index[-1], 'Year'] = 'Total'

    
    # Reorder columns to match Table S2 structure
    column_order = [
        'Year', 'Overall', 'Women', 'Men',
        'White', 'Black/AA', 'A/PI', 'AI/AN',
        '25-39 years', '40-54 years', '55-69 years', 
        '70-84 years', '85+ years', 'Population'
    ]
    
    return final_table[column_order]

# Generate and save the table
table_s2 = create_table_s2()
table_s2.to_csv("T1_SU_HD_Demographics_2005_2015.csv", index=False)


### **Table 2: Absolute Number of overall SU, overall HD, overall SU categories and CVD+ SU categories Related Deaths in the United States, 2005-2015**

In [9]:
import pandas as pd
import numpy as np

# Define substance categories with their ICD-10 codes
substance_categories = {
    'Opioids': ['F11', 'R781', 'T400', 'T401', 'T402', 'T403', 'T404', 'T406', 'X62', 'Y12'],
    'Cannabis': ['F12', 'T407'],
    'Cocaine': ['F14', 'R782', 'T405'],
    'Sedatives/Hypnotics': ['F13', 'T423', 'T424', 'T426', 'T427', 'X61', 'Y11'],
    'Alcohol': ['E244', 'F10', 'G312', 'G621', 'G721', 'I426', 'K70', 'K852', 'K860', 'R780', 'T51', 'X65', 'Y15'],
    'Stimulants': ['F15', 'T436']
}

# Load datasets
su_mortality = pd.read_csv('SU_mortality_final.csv', low_memory=False)
hd_mortality = pd.read_csv('HD_mortality_final.csv', low_memory=False)
su_hd_mortality = pd.read_csv('SU_HD_mortality_final.csv', low_memory=False)
population_data = pd.read_csv('population_data_v1.csv', low_memory=False)

# Filter population data for ages 25+
population_data = population_data[population_data['age_recode_custom'] > 0]

# Calculate total population per year
yearly_population = population_data.groupby('Year')['Population'].sum().reset_index()
yearly_population.rename(columns={'Year': 'year'}, inplace=True)

# Function to check if any code in a row matches a substance category
def check_substance(row, substance_codes):
    # Get all ICD codes from the row
    icd_columns = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]
    codes = [str(row[col]) for col in icd_columns if pd.notna(row[col])]
    
    # Check if any code starts with any substance code
    for code in codes:
        for substance_code in substance_codes:
            if str(code).startswith(substance_code):
                return True
    return False

# Create result dataframe
years_range = range(2005, 2016)
result = pd.DataFrame(index=years_range)

# Count All SU and All HD deaths per year
all_su_counts = su_mortality.groupby('current_data_year').size()
all_hd_counts = hd_mortality.groupby('current_data_year').size()
result['All SU'] = all_su_counts
result['All HD'] = all_hd_counts

# Count SU+HD deaths by substance category
for substance, codes in substance_categories.items():
    # For SU+HD combined
    substance_counts = []
    for year in years_range:
        year_data = su_hd_mortality[su_hd_mortality['current_data_year'] == year]
        count = sum(year_data.apply(lambda row: check_substance(row, codes), axis=1))
        substance_counts.append(count)
    result[f'{substance} HD'] = substance_counts
    
    # For SU only
    substance_counts = []
    for year in years_range:
        year_data = su_mortality[su_mortality['current_data_year'] == year]
        count = sum(year_data.apply(lambda row: check_substance(row, codes), axis=1))
        substance_counts.append(count)
    result[substance] = substance_counts

# Add population data
result = result.reset_index().rename(columns={'index': 'Year'})
result = pd.merge(result, yearly_population, left_on='Year', right_on='year', how='left')
result.drop('year', axis=1, inplace=True)

# Calculate total row
total_row = pd.DataFrame({
    'Year': ['Total'],
    'All SU': [result['All SU'].sum()],
    'All HD': [result['All HD'].sum()],
    'Alcohol HD': [result['Alcohol HD'].sum()],
    'Opioids HD': [result['Opioids HD'].sum()],
    'Cocaine HD': [result['Cocaine HD'].sum()],
    'Sedatives/Hypnotics HD': [result['Sedatives/Hypnotics HD'].sum()],
    'Stimulants HD': [result['Stimulants HD'].sum()],
    'Cannabis HD': [result['Cannabis HD'].sum()],
    'Alcohol': [result['Alcohol'].sum()],
    'Opioids': [result['Opioids'].sum()],
    'Cocaine': [result['Cocaine'].sum()],
    'Sedatives/Hypnotics': [result['Sedatives/Hypnotics'].sum()],
    'Stimulants': [result['Stimulants'].sum()],
    'Cannabis': [result['Cannabis'].sum()],
    'Population': [result['Population'].sum()]
})

# Append total row to result
table_s3 = pd.concat([result, total_row], ignore_index=True)

# Reorder columns to match Table S3
column_order = ['Year', 'All SU', 'All HD', 'Alcohol HD', 'Opioids HD', 'Cocaine HD', 
                'Sedatives/Hypnotics HD', 'Stimulants HD', 'Cannabis HD', 
                'Alcohol', 'Opioids', 'Cocaine', 'Sedatives/Hypnotics', 
                'Stimulants', 'Cannabis', 'Population']
table_s3 = table_s3[column_order]

# Display and save the result
print(table_s3)
table_s3.to_csv('T2_SU_CVD_Mortality_Counts_2005_2015.csv.csv', index=False)


     Year  All SU    All HD  Alcohol HD  Opioids HD  Cocaine HD  \
0    2005   71908   1126312       11938        2481        2613   
1    2006   77210   1106274       12563        2856        3090   
2    2007   79226   1095280       12924        3042        3034   
3    2008   80635   1111057       13568        3223        2535   
4    2009   82272   1089759       13994        3765        2213   
5    2010   85732   1099626       14756        3789        2229   
6    2011   90850   1112176       15279        4095        2327   
7    2012   93214   1119006       16062        4166        2171   
8    2013   98380   1148304       17505        4435        2392   
9    2014  104796   1157166       18319        4989        2497   
10   2015  114271   1199164       19808        5211        2677   
11  Total  978494  12364124      166716       42052       27778   

    Sedatives/Hypnotics HD  Stimulants HD  Cannabis HD  Alcohol  Opioids  \
0                      617            740           

### **Table 3: Substance use And Heart Disease related age-adjusted mortality rates per 100,000 in the United States, 2005 to 2015.**

In [26]:
#Standard Population Weights (2000 US Standard Population)
# 1 = 25-39, 2 = 40-54, 3 = 55-69, 4 = 70-84, 5 = 85+
intervals = ["25-39", "40-54", "55-69", "70-87", "85+"]
aw_1 = 0.064530 + 0.071044 + 0.080762
aw_2 = 0.081851 + 0.072118 + 0.062716
aw_3 = 0.048454 + 0.038793 + 0.034264
aw_4 = 0.031773 + 0.027000 + 0.017842
aw_5 = 0.015508

print("The SPW per our age intervals are as follows:")

#Renormalizing our weights to equal 1
aw_1 = round(aw_1/0.646655, 6)
aw_2 = round(aw_2/0.646655, 6)
aw_3 = round(aw_3/0.646655, 6)
aw_4 = round(aw_4/0.646655, 6)
aw_5 = round(aw_5/0.646655, 6)
weight = [aw_1, aw_2, aw_3, aw_4, aw_5]
x = 0
for interval in intervals:
    print(f"{interval} weight = {weight[x]}")
    x += 1

The SPW per our age intervals are as follows:
25-39 weight = 0.334546
40-54 weight = 0.335086
55-69 weight = 0.187907
70-87 weight = 0.118479
85+ weight = 0.023982


In [27]:
import pandas as pd

su_data = pd.read_csv('SU_mortality_final.csv', low_memory = False)
hd_data = pd.read_csv('HD_mortality_final.csv', low_memory = False)
pop_data = pd.read_csv('population_data_v1.csv', low_memory = False)

std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years
}

# Function to calculate AAMR
def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result


# Calculate AAMR for SU and HD
su_aamr = calculate_aamr(su_data, pop_data, std_weights)
hd_aamr = calculate_aamr(hd_data, pop_data, std_weights)

# Combine results into a single table
result_table = pd.merge(su_aamr, hd_aamr, on='Year', suffixes=(' (SU)', ' (HD)'))
print(result_table)

# Save to CSV
result_table.to_csv('T3_SU_HD_AAMR_2005_2015.csv', index=False)


     Year  AAMR (SU)   AAMR (HD)
0    2005  36.747214  582.439821
1    2006  38.974507  560.201187
2    2007  39.431919  543.329992
3    2008  39.584410  539.710370
4    2009  39.846737  518.001806
5    2010  40.906093  510.473185
6    2011  42.878493  504.679008
7    2012  43.401941  494.605606
8    2013  45.222921  495.161323
9    2014  47.664900  487.630457
10   2015  51.492468  493.419706
11  Total  42.574173  519.018372


### **Table 4: Substance use + Heart Disease related Deaths Stratified by Heart Disease conditions AAMR in the United States, 2005-2015.**

In [28]:
#Extract Ischemic Heart Disease (I20-25)
import pandas as pd

df = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)

codes = ['I20', "I21", "I22", "I23", "I24", "I25"]

# All columns to check
cols = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Filter only columns that exist in the DataFrame
cols = [col for col in cols if col in df.columns]

# Create a boolean DataFrame for startswith across all relevant columns
mask = df[cols].astype(str).apply(lambda col: col.str.startswith(tuple(codes)))

# Keep rows where any column matches
filtered_df = df[mask.any(axis=1)]

filtered_df.to_csv('SU_HD_con_ischemic.csv', index=False)

In [29]:
#Extract Heart Failure/Cardiomyopathy (I50, I40-43)
import pandas as pd

df = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)

codes = ["I50", "I40", "I41", "I42", "I43"]

# All columns to check
cols = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Filter only columns that exist in the DataFrame
cols = [col for col in cols if col in df.columns]

# Create a boolean DataFrame for startswith across all relevant columns
mask = df[cols].astype(str).apply(lambda col: col.str.startswith(tuple(codes)))

# Keep rows where any column matches
filtered_df = df[mask.any(axis=1)]

filtered_df.to_csv('SU_HD_con_failure_cardiomyopathy.csv', index=False)

In [30]:
#Extract Infective Endocarditis (I33-I38)
import pandas as pd

df = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)

codes = ["I33", "I34", "I35", "I36", "I37", "I38"]

# All columns to check
cols = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Filter only columns that exist in the DataFrame
cols = [col for col in cols if col in df.columns]

# Create a boolean DataFrame for startswith across all relevant columns
mask = df[cols].astype(str).apply(lambda col: col.str.startswith(tuple(codes)))

# Keep rows where any column matches
filtered_df = df[mask.any(axis=1)]

filtered_df.to_csv('SU_HD_con_infective.csv', index=False)

In [31]:
#Extract Hypertensive Heart Disease (I11 and I13)
import pandas as pd

df = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)

codes = ["I11", "I13"]

# All columns to check
cols = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Filter only columns that exist in the DataFrame
cols = [col for col in cols if col in df.columns]

# Create a boolean DataFrame for startswith across all relevant columns
mask = df[cols].astype(str).apply(lambda col: col.str.startswith(tuple(codes)))

# Keep rows where any column matches
filtered_df = df[mask.any(axis=1)]

filtered_df.to_csv('SU_HD_con_hypertensive.csv', index=False)

In [36]:
ischemic_data = pd.read_csv('SU_HD_con_ischemic.csv', low_memory = False)
failure_data = pd.read_csv('SU_HD_con_failure_cardiomyopathy.csv', low_memory = False)
infective_data = pd.read_csv('SU_HD_con_infective.csv', low_memory = False)
hyper_data = pd.read_csv('SU_HD_con_hypertensive.csv', low_memory = False)
pop_data = pd.read_csv('population_data_v1.csv', low_memory = False)

std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}
#AAMR calcualtor from table 3
def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

ischemic_aamr = calculate_aamr(ischemic_data, pop_data, std_weights)
failure_aamr = calculate_aamr(failure_data, pop_data, std_weights)
infective_aamr = calculate_aamr(infective_data, pop_data, std_weights)
hyper_aamr = calculate_aamr(hyper_data, pop_data, std_weights)

# Rename columns first
ischemic_aamr = ischemic_aamr.rename(columns={'AAMR': 'AAMR (Ischemic Heart Disease)'})
failure_aamr = failure_aamr.rename(columns={'AAMR': 'AAMR (Heart Failure/Cardiomyopathy)'})
infective_aamr = infective_aamr.rename(columns={'AAMR': 'AAMR (Infective Endocarditis)'})
hyper_aamr = hyper_aamr.rename(columns={'AAMR': 'AAMR (Hypertensive Heart Disease)'})

# Merge sequentially
result_table = pd.merge(ischemic_aamr, failure_aamr, on='Year')
result_table = pd.merge(result_table, infective_aamr, on='Year')
result_table = pd.merge(result_table, hyper_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T4_SU_HD_CAT_AAMR_2005_2015.csv', index=False)


     Year  AAMR (Ischemic Heart Disease)  AAMR (Heart Failure/Cardiomyopathy)  \
0    2005                       4.109294                             1.796515   
1    2006                       4.399306                             1.775867   
2    2007                       4.547318                             1.785533   
3    2008                       4.465998                             1.773597   
4    2009                       4.502114                             1.743795   
5    2010                       4.662350                             1.707629   
6    2011                       4.790770                             1.803658   
7    2012                       4.881008                             1.832352   
8    2013                       5.138469                             1.913406   
9    2014                       5.258858                             1.958760   
10   2015                       5.487497                             2.101280   
11  Total                   

### **Table 5:  Overall and Sex-Stratified Substance use + Heart Disease related Age-Adjusted Mortality Rates per 100,000 in the United States, 2005-2015**

In [38]:
import pandas as pd 
df = pd.read_csv("SU_HD_mortality_final.csv", low_memory = False)
#Extract females only
df = df[df["sex"] == "F"]
df.to_csv("SU_HD_mortality_female.csv", index = False)
#Extract males only
df = pd.read_csv("SU_HD_mortality_final.csv", low_memory = False)
df = df[df["sex"] == "M"]
df.to_csv("SU_HD_mortality_male.csv", index = False)

In [40]:
#Run AAMR function
overall = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)
male = pd.read_csv("SU_HD_mortality_male.csv", low_memory=False)
female = pd.read_csv("SU_HD_mortality_female.csv", low_memory=False)
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)

# Filter population data by sex
male_pop = pop_data[pop_data['Sex'] == 'M']
female_pop = pop_data[pop_data['Sex'] == 'F']

std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}

def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

overall_aamr = calculate_aamr(overall, pop_data, std_weights)
male_aamr = calculate_aamr(male, male_pop, std_weights)
female_aamr = calculate_aamr(female, female_pop, std_weights)

# Rename columns first
overall_aamr = overall_aamr.rename(columns={'AAMR': 'AAMR (Overall)'})
male_aamr = male_aamr.rename(columns={'AAMR': 'AAMR (Male)'})
female_aamr = female_aamr.rename(columns={'AAMR': 'AAMR (Female)'})

# Merge sequentially
result_table = pd.merge(overall_aamr, male_aamr, on='Year')
result_table = pd.merge(result_table, female_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T5_SU_HD_MF_AAMR_2005_2015.csv', index=False)

     Year  AAMR (Overall)  AAMR (Male)  AAMR (Female)
0    2005        9.663843    15.765241       4.158419
1    2006       10.271466    16.540979       4.558432
2    2007       10.483638    16.917839       4.617902
3    2008       10.521209    16.858448       4.742009
4    2009       10.775494    17.036807       5.056762
5    2010       11.036064    17.415595       5.201272
6    2011       11.472091    18.031798       5.445256
7    2012       11.716321    18.387418       5.587831
8    2013       12.565926    19.590512       6.095664
9    2014       13.061156    20.307524       6.382779
10   2015       13.861414    21.554560       6.770063
11  Total       11.469659    18.141035       5.362843


### **Table 6:  Substance use + Heart Disease related age-adjusted mortality rates per 100,000 stratified by race/ethnicity in the United States, 2005 to 2015**

In [41]:
#Run AAMR function
suhd = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)

white_mort = suhd[suhd['race_recode_5'] == 1]
black_mort = suhd[suhd['race_recode_5'] == 2]
ai_mort = suhd[suhd['race_recode_5'] == 3]
asian_mort = suhd[suhd['race_recode_5'] == 4]

# Filter population data by sex
white_pop = pop_data[pop_data['race_recode_5'] == 1]
black_pop = pop_data[pop_data['race_recode_5'] == 2]
ai_pop = pop_data[pop_data['race_recode_5'] == 3]
asian_pop = pop_data[pop_data['race_recode_5'] == 4]

std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}

def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

white_aamr = calculate_aamr(white_mort, white_pop, std_weights)
black_aamr = calculate_aamr(black_mort, black_pop, std_weights)
ai_aamr = calculate_aamr(ai_mort, ai_pop, std_weights)
asian_aamr = calculate_aamr(asian_mort, asian_pop, std_weights)

# Rename columns first
white_aamr = white_aamr.rename(columns={'AAMR': 'AAMR (White)'})
black_aamr = black_aamr.rename(columns={'AAMR': 'AAMR (Black/AA)'})
ai_aamr = ai_aamr.rename(columns={'AAMR': 'AAMR (AI/AN)'})
asian_aamr = asian_aamr.rename(columns={'AAMR': 'AAMR (A/PI)'})

# Merge sequentially
result_table = pd.merge(white_aamr, black_aamr, on='Year')
result_table = pd.merge(result_table, ai_aamr, on='Year')
result_table = pd.merge(result_table, asian_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T6_SU_HD_RACE_AAMR_2005_2015.csv', index=False)

     Year  AAMR (White)  AAMR (Black/AA)  AAMR (AI/AN)  AAMR (A/PI)
0    2005      9.184766        15.613226     16.783282     2.349219
1    2006      9.826236        16.058713     18.183216     2.622867
2    2007     10.136379        15.768587     17.824188     2.833851
3    2008     10.443854        14.125383     17.364444     2.498841
4    2009     10.750619        14.058798     17.162363     2.851421
5    2010     11.158678        13.566126     18.838988     2.399309
6    2011     11.596347        14.126917     19.186064     2.767974
7    2012     11.922718        13.940362     20.695414     2.715885
8    2013     12.810438        14.845923     21.746755     2.851783
9    2014     13.368117        15.038254     24.181529     3.261474
10   2015     14.197888        16.089611     25.175723     3.269927
11  Total     11.456017        14.891438     19.964840     2.794964


### **Table 7: Substance use + Heart Disease related crude mortality rates per 100,000 stratified by age groups in the United States, 2005 to 2015.**

In [44]:
# Run crude mortality rate calculation
suhd = pd.read_csv("SU_HD_mortality_final.csv", low_memory=False)
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)

mort_1 = suhd[suhd['age_recode_custom'] == 1]
mort_2 = suhd[suhd['age_recode_custom'] == 2]
mort_3 = suhd[suhd['age_recode_custom'] == 3]
mort_4 = suhd[suhd['age_recode_custom'] == 4]
mort_5 = suhd[suhd['age_recode_custom'] == 5]

# Filter population data by age
pop_1 = pop_data[pop_data['age_recode_custom'] == 1]
pop_2 = pop_data[pop_data['age_recode_custom'] == 2]
pop_3 = pop_data[pop_data['age_recode_custom'] == 3]
pop_4 = pop_data[pop_data['age_recode_custom'] == 4]
pop_5 = pop_data[pop_data['age_recode_custom'] == 5]

def calculate_crude_rate(mortality_data, population_data):
    # Group mortality data by year, count deaths
    deaths_by_year = mortality_data.groupby(['current_data_year']).size().reset_index(name='deaths')
    
    # Group population data by year
    pop_by_year = population_data.groupby(['Year'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year = pop_by_year.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year, pop_by_year, on='current_data_year')
    
    # Calculate crude rates per 100,000
    merged_data['crude_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Prepare yearly results
    yearly_rates = merged_data[['current_data_year', 'crude_rate']]
    yearly_rates.columns = ['Year', 'Rate']
    
    # Calculate overall crude rate for entire period
    total_deaths = mortality_data.shape[0]
    total_population = population_data['Population'].sum()
    overall_rate = (total_deaths / total_population) * 100000
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'Rate': [overall_rate]})
    final_result = pd.concat([yearly_rates, overall_row], ignore_index=True)
    
    return final_result

one_rate = calculate_crude_rate(mort_1, pop_1)
two_rate = calculate_crude_rate(mort_2, pop_2)
three_rate = calculate_crude_rate(mort_3, pop_3)
four_rate = calculate_crude_rate(mort_4, pop_4)
five_rate = calculate_crude_rate(mort_5, pop_5)

# Rename columns
one_rate = one_rate.rename(columns={'Rate': 'Rate (Age 25-39 Years)'})
two_rate = two_rate.rename(columns={'Rate': 'Rate (Age 40-54 Years)'})
three_rate = three_rate.rename(columns={'Rate': 'Rate (Age 55-69 Years)'})
four_rate = four_rate.rename(columns={'Rate': 'Rate (Age 70-84 Years)'})
five_rate = five_rate.rename(columns={'Rate': 'Rate (Age 85+ Years)'})

# Merge sequentially
result_table = pd.merge(one_rate, two_rate, on='Year')
result_table = pd.merge(result_table, three_rate, on='Year')
result_table = pd.merge(result_table, four_rate, on='Year')
result_table = pd.merge(result_table, five_rate, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T7_SU_HD_AGE_CRUDE_RATES_2005_2015.csv', index=False)


     Year  Rate (Age 25-39 Years)  Rate (Age 40-54 Years)  \
0    2005                3.453369               13.053123   
1    2006                3.949522               14.177741   
2    2007                3.716378               14.262848   
3    2008                3.778783               14.107897   
4    2009                3.950604               14.310425   
5    2010                4.135723               14.246439   
6    2011                4.357424               14.899087   
7    2012                4.307324               14.891213   
8    2013                4.508613               15.913239   
9    2014                4.936945               16.230173   
10   2015                5.301357               16.792486   
11  Total                4.228345               14.797471   

    Rate (Age 55-69 Years)  Rate (Age 70-84 Years)  Rate (Age 85+ Years)  
0                14.599174               10.263923              7.308292  
1                15.194201               10.094285      

### **Table 8: Substance use categories + Heart Disease related age-adjusted mortality rates per 100,000 in the United States, 2005 to 2015.**

In [47]:
import pandas as pd

substance_categories = {
    'Opioids': ['F11', 'R781', 'T400', 'T401', 'T402', 'T403', 'T404', 'T406', 'X62', 'Y12'],
    'Cannabis': ['F12', 'T407'],
    'Cocaine': ['F14', 'R782', 'T405'],
    'Sedatives/Hypnotics': ['F13', 'T423', 'T424', 'T426', 'T427', 'X61', 'Y11'],
    'Alcohol': ['E244', 'F10', 'G312', 'G621', 'G721', 'I426', 'K70', 'K852', 'K860', 'R780', 'T51', 'X65', 'Y15'],
    'Stimulants': ['F15', 'T436']
}
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)
mortality_data = pd.read_csv('SU_HD_mortality_final.csv', low_memory=False)

icd_columns = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Create dataframes for each substance category
category_dfs = {}
for category, codes in substance_categories.items():
    # Use a Series for the mask instead of a DataFrame
    mask = pd.Series(False, index=mortality_data.index)
    
    for col in icd_columns:
        if col in mortality_data.columns:  # Check if column exists
            for code in codes:
                # Directly update the mask Series
                mask = mask | mortality_data[col].astype(str).str.startswith(code, na=False)
    
    category_dfs[category] = mortality_data[mask]


std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}

def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

opioids_aamr = calculate_aamr(category_dfs['Opioids'], pop_data, std_weights)
cannabis_aamr = calculate_aamr(category_dfs['Cannabis'], pop_data, std_weights)
cocaine_aamr = calculate_aamr(category_dfs['Cocaine'], pop_data, std_weights)
sedatives_hypnotics_aamr = calculate_aamr(category_dfs['Sedatives/Hypnotics'], pop_data, std_weights)
alcohol_aamr = calculate_aamr(category_dfs['Alcohol'], pop_data, std_weights)
stimulants_aamr = calculate_aamr(category_dfs['Stimulants'], pop_data, std_weights)

# Rename columns first
opioids_aamr = opioids_aamr.rename(columns={'AAMR': 'Opioids'})
cannabis_aamr = cannabis_aamr.rename(columns={'AAMR': 'Cannabis'})
cocaine_aamr = cocaine_aamr.rename(columns={'AAMR': 'Cocaine'})
sedatives_hypnotics_aamr = sedatives_hypnotics_aamr.rename(columns={'AAMR': 'Sedatives/Hypnotics'})
alcohol_aamr = alcohol_aamr.rename(columns={'AAMR': 'Alcohol'})
stimulants_aamr = stimulants_aamr.rename(columns={'AAMR': 'Stimulants'})

# Merge sequentially
result_table = pd.merge(opioids_aamr, cannabis_aamr, on='Year')
result_table = pd.merge(result_table, cocaine_aamr, on='Year')
result_table = pd.merge(result_table, sedatives_hypnotics_aamr, on='Year')
result_table = pd.merge(result_table, alcohol_aamr, on='Year')
result_table = pd.merge(result_table, stimulants_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T8_SU_HD_SUBSTANCE_AAMR_2005_2015.csv', index=False)

     Year   Opioids  Cannabis   Cocaine  Sedatives/Hypnotics   Alcohol  \
0    2005  1.280179  0.027961  1.340760             0.316373  5.969609   
1    2006  1.462440  0.036397  1.568087             0.381440  6.166864   
2    2007  1.533604  0.040027  1.515036             0.387596  6.231236   
3    2008  1.604795  0.039255  1.251586             0.464698  6.433140   
4    2009  1.850442  0.042331  1.076210             0.586945  6.520600   
5    2010  1.846737  0.040594  1.061842             0.623637  6.739757   
6    2011  1.973105  0.050969  1.095737             0.666548  6.851431   
7    2012  1.977124  0.067380  1.009732             0.646962  7.058946   
8    2013  2.080288  0.062829  1.101335             0.704524  7.551779   
9    2014  2.324665  0.084375  1.129319             0.770151  7.777839   
10   2015  2.390843  0.087895  1.194676             0.749201  8.272903   
11  Total  1.861821  0.053349  1.216853             0.577233  6.908486   

    Stimulants  
0     0.383272  
1  

### **Table 9: Substance use categories related age-adjusted mortality rates per 100,000 in the United States, 2005 to 2015.**

In [49]:
import pandas as pd

substance_categories = {
    'Opioids': ['F11', 'R781', 'T400', 'T401', 'T402', 'T403', 'T404', 'T406', 'X62', 'Y12'],
    'Cannabis': ['F12', 'T407'],
    'Cocaine': ['F14', 'R782', 'T405'],
    'Sedatives/Hypnotics': ['F13', 'T423', 'T424', 'T426', 'T427', 'X61', 'Y11'],
    'Alcohol': ['E244', 'F10', 'G312', 'G621', 'G721', 'I426', 'K70', 'K852', 'K860', 'R780', 'T51', 'X65', 'Y15'],
    'Stimulants': ['F15', 'T436']
}
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)
mortality_data = pd.read_csv('SU_mortality_final.csv', low_memory=False)

icd_columns = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Create dataframes for each substance category
category_dfs = {}
for category, codes in substance_categories.items():
    # Use a Series for the mask instead of a DataFrame
    mask = pd.Series(False, index=mortality_data.index)
    
    for col in icd_columns:
        if col in mortality_data.columns:  # Check if column exists
            for code in codes:
                # Directly update the mask Series
                mask = mask | mortality_data[col].astype(str).str.startswith(code, na=False)
    
    category_dfs[category] = mortality_data[mask]


std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}

def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

opioids_aamr = calculate_aamr(category_dfs['Opioids'], pop_data, std_weights)
cannabis_aamr = calculate_aamr(category_dfs['Cannabis'], pop_data, std_weights)
cocaine_aamr = calculate_aamr(category_dfs['Cocaine'], pop_data, std_weights)
sedatives_hypnotics_aamr = calculate_aamr(category_dfs['Sedatives/Hypnotics'], pop_data, std_weights)
alcohol_aamr = calculate_aamr(category_dfs['Alcohol'], pop_data, std_weights)
stimulants_aamr = calculate_aamr(category_dfs['Stimulants'], pop_data, std_weights)

# Rename columns first
opioids_aamr = opioids_aamr.rename(columns={'AAMR': 'Opioids'})
cannabis_aamr = cannabis_aamr.rename(columns={'AAMR': 'Cannabis'})
cocaine_aamr = cocaine_aamr.rename(columns={'AAMR': 'Cocaine'})
sedatives_hypnotics_aamr = sedatives_hypnotics_aamr.rename(columns={'AAMR': 'Sedatives/Hypnotics'})
alcohol_aamr = alcohol_aamr.rename(columns={'AAMR': 'Alcohol'})
stimulants_aamr = stimulants_aamr.rename(columns={'AAMR': 'Stimulants'})

# Merge sequentially
result_table = pd.merge(opioids_aamr, cannabis_aamr, on='Year')
result_table = pd.merge(result_table, cocaine_aamr, on='Year')
result_table = pd.merge(result_table, sedatives_hypnotics_aamr, on='Year')
result_table = pd.merge(result_table, alcohol_aamr, on='Year')
result_table = pd.merge(result_table, stimulants_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T9_SU_SUBSTANCE_AAMR_2005_2015.csv', index=False)

     Year    Opioids  Cannabis   Cocaine  Sedatives/Hypnotics    Alcohol  \
0    2005   7.882922  0.095496  4.576362             2.259943  21.201465   
1    2006   9.011999  0.108289  5.269495             2.595565  21.599760   
2    2007   9.283698  0.119683  4.719064             2.897625  21.740482   
3    2008   9.695191  0.119888  3.763927             3.138945  22.345228   
4    2009  10.152049  0.121667  3.218863             3.551911  22.368765   
5    2010  10.338886  0.142086  3.048659             3.968523  22.953308   
6    2011  11.112989  0.182983  3.308096             4.219356  23.590894   
7    2012  11.259204  0.207634  3.020577             4.085355  24.183105   
8    2013  12.136756  0.223058  3.311886             4.367910  25.025609   
9    2014  13.769079  0.246906  3.558510             4.865992  26.187556   
10   2015  15.778008  0.322196  4.208525             5.363519  27.693083   
11  Total  11.011680  0.173336  3.825255             3.780229  23.641099   

    Stimula

### **CHUNK THIS: Table 10: Heart Disease related Deaths Stratified by Heart Disease conditions AAMR in the United States, 2005-2015.**

In [ ]:
import pandas as pd

substance_categories = {
    'Ischemic': ['I20', "I21", "I22", "I23", "I24", "I25"],
    'Heart Failure/Cardiomyopathy': ["I50", "I40", "I41", "I42", "I43"],
    'Infective Endocarditis': ["I33", "I34", "I35", "I36", "I37", "I38"],
    'Hypertensive Heart Disease': ["I11", "I13"]
}
pop_data = pd.read_csv('population_data_v1.csv', low_memory=False)
mortality_data = pd.read_csv('HD_mortality_final.csv', low_memory=False)

icd_columns = ['icd_code_10th_revision'] + [f'record_condition_{i}' for i in range(1, 21)]

# Create dataframes for each substance category
category_dfs = {}
for category, codes in substance_categories.items():
    # Use a Series for the mask instead of a DataFrame
    mask = pd.Series(False, index=mortality_data.index)
    
    for col in icd_columns:
        if col in mortality_data.columns:  # Check if column exists
            for code in codes:
                # Directly update the mask Series
                mask = mask | mortality_data[col].astype(str).str.startswith(code, na=False)
    
    category_dfs[category] = mortality_data[mask]


std_weights = {
    1: 0.334546,  # 25-39 years
    2: 0.335086,  # 40-54 years
    3: 0.187907,  # 55-69 years
    4: 0.118479,  # 70-84 years
    5: 0.023982   # 85+ years 
}

def calculate_aamr(mortality_data, population_data, weights):
    population_data = population_data[population_data['age_recode_custom'] > 0]
    # Group mortality data by year and age group, count deaths
    deaths_by_year_age = mortality_data.groupby(['current_data_year', 'age_recode_custom']).size().reset_index(name='deaths')
    
    # Group population data by year and age group
    pop_by_year_age = population_data.groupby(['Year', 'age_recode_custom'])['Population'].sum().reset_index()
    
    # Rename Year column in population data to match mortality data for merging
    pop_by_year_age = pop_by_year_age.rename(columns={'Year': 'current_data_year'})
    
    # Merge deaths and population data
    merged_data = pd.merge(deaths_by_year_age, pop_by_year_age, 
                          on=['current_data_year', 'age_recode_custom'])
    
    # Calculate age-specific rates per 100,000
    merged_data['age_specific_rate'] = (merged_data['deaths'] / merged_data['Population']) * 100000
    
    # Apply weights to get weighted rates
    merged_data['weighted_rate'] = merged_data.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates by year to get yearly AAMR
    yearly_aamr = merged_data.groupby('current_data_year')['weighted_rate'].sum().reset_index()
    yearly_aamr.columns = ['Year', 'AAMR']
    
    # Calculate overall AAMR for entire period
    # First, get total deaths and population by age group for entire period
    total_deaths_by_age = mortality_data.groupby('age_recode_custom').size().reset_index(name='deaths')
    
    # For population, we need to sum across years for each age group
    total_pop_by_age = population_data.groupby('age_recode_custom')['Population'].sum().reset_index()
    
    # Merge total deaths and population
    total_merged = pd.merge(total_deaths_by_age, total_pop_by_age, on='age_recode_custom')
    
    # Calculate overall age-specific rates
    total_merged['age_specific_rate'] = (total_merged['deaths'] / total_merged['Population']) * 100000
    
    # Apply weights
    total_merged['weighted_rate'] = total_merged.apply(
        lambda row: row['age_specific_rate'] * weights[row['age_recode_custom']], axis=1)
    
    # Sum weighted rates to get overall AAMR
    overall_aamr = total_merged['weighted_rate'].sum()
    
    # Add overall row to yearly results
    overall_row = pd.DataFrame({'Year': ['Total'], 'AAMR': [overall_aamr]})
    final_result = pd.concat([yearly_aamr, overall_row], ignore_index=True)
    
    return final_result

ischemic_aamr = calculate_aamr(category_dfs['Ischemic'], pop_data, std_weights)
hf_aamr = calculate_aamr(category_dfs['Heart Failure/Cardiomyopathy'], pop_data, std_weights)
infective_aamr = calculate_aamr(category_dfs['Infective Endocarditis'], pop_data, std_weights)
hypertensive_aamr = calculate_aamr(category_dfs['Hypertensive Heart Disease'], pop_data, std_weights)


# Rename columns first
ischemic_aamr = ischemic_aamr.rename(columns={'AAMR': 'Ischemic Heart Disease'})
hf_aamr = hf_aamr.rename(columns={'AAMR': 'Heart Failure/Cardiomyopathy'})
infective_aamr = infective_aamr.rename(columns={'AAMR': 'Infective Endocarditis'})
hypertensive_aamr = hypertensive_aamr.rename(columns={'AAMR': 'Hypertensive Heart Disease'})


# Merge sequentially
result_table = pd.merge(ischemic_aamr, hf_aamr, on='Year')
result_table = pd.merge(result_table, infective_aamr, on='Year')
result_table = pd.merge(result_table, hypertensive_aamr, on='Year')

print(result_table)

# Save to CSV
result_table.to_csv('T10_HD_CAT_AAMR_2005_2015.csv', index=False)

### **Final**

In [2]:
import pandas as pd
from math import sqrt

t1 = pd.read_csv('T1_SU_HD_Demographics_2005_2015.csv')
t2 = pd.read_csv('T2_SU_CVD_Mortality_Counts_2005_2015.csv')
t3 = pd.read_csv('T3_SU_HD_AAMR_2005_2015.csv')
t5 = pd.read_csv('T5_SU_HD_MF_AAMR_2005_2015.csv')
t6 = pd.read_csv('T6_SU_HD_RACE_AAMR_2005_2015.csv')
t7 = pd.read_csv('T7_SU_HD_AGE_CRUDE_RATES_2005_2015.csv')
t8 = pd.read_csv('T8_SU_HD_SUBSTANCE_AAMR_2005_2015.csv')
t9 = pd.read_csv('T9_SU_SUBSTANCE_AAMR_2005_2015.csv')

def confidence_interval(mortality_rate, deaths):
    se = mortality_rate/sqrt(deaths)
    upper_ci = mortality_rate + (1.96 * se)
    lower_ci = mortality_rate - (1.96 * se)
    return upper_ci, lower_ci


result = pd.DataFrame(columns=[
    'Category', 'Year 2005', '95% CI (%)', 'Year 2015', '95% CI (%)', 
    'Overall from 2005-2015', '95% CI (%)', 'AAPC (%)', '95% CI (%)'])

def add_mortality_data(rate_column, death_column, category_name):
    # Get the corresponding DataFrame for each column to access the Year
    rate_df = rate_column.reset_index()
    death_df = death_column.reset_index()
    
    # Extract rates for 2005, 2015, and Total
    rate_2005 = rate_column.iloc[0]  # 2005 is at index 0
    rate_2015 = rate_column.iloc[10]  # 2015 is at index 10
    rate_overall = rate_column.iloc[11]  # Total is at index 11
    
    # Extract deaths for 2005, 2015, and Total
    death_2005 = death_column.iloc[0]  # 2005 is at index 0
    death_2015 = death_column.iloc[10]  # 2015 is at index 10
    death_overall = death_column.iloc[11]  # Total is at index 11
  
    death_2005 = round(death_2005, 2)
    death_2015 = round(death_2015, 2)
    death_overall = round(death_overall, 2)
    
    # Calculate confidence intervals
    lower_ci_2005, upper_ci_2005 = confidence_interval(rate_2005, death_2005)
    lower_ci_2015, upper_ci_2015 = confidence_interval(rate_2015, death_2015)
    lower_ci_overall, upper_ci_overall = confidence_interval(rate_overall, death_overall)

    rate_2005 = round(rate_2005, 2)
    rate_2015 = round(rate_2015, 2)
    rate_overall = round(rate_overall, 2)

    death_2005 = round(death_2005, 2)
    death_2015 = round(death_2015, 2)
    death_overall = round(death_overall, 2)
    
    # Format confidence intervals
    ci_2005 = f"{upper_ci_2005:.2f} to {lower_ci_2005:.2f} "
    ci_2015 = f"{upper_ci_2015:.2f} to {lower_ci_2015:.2f}"
    ci_overall = f"{upper_ci_overall:.2f} to {lower_ci_overall:.2f}"
    
    # Create new row
    new_row = [
        category_name, 
        rate_2005, 
        ci_2005, 
        rate_2015, 
        ci_2015, 
        rate_overall, 
        ci_overall, 
        None, 
        None
    ]
    
    # Add row to result DataFrame
    result.loc[len(result)] = new_row

#SU-related AAMR
add_mortality_data(t3["AAMR (SU)"],t2["All SU"] , "SU‐related mortality")
#SU AAMR by Drug Category
add_mortality_data(t9["Alcohol"],t2["Alcohol"] , "Alcohol")
add_mortality_data(t9["Cannabis"],t2["Cannabis"] , "Cannabis")
add_mortality_data(t9["Cocaine"],t2["Cocaine"] , "Cocaine")
add_mortality_data(t9["Opioids"],t2["Opioids"] , "Opioid")
add_mortality_data(t9["Sedatives/Hypnotics"],t2["Sedatives/Hypnotics"] , "Sedative-hypnotics")
add_mortality_data(t9["Stimulants"],t2["Stimulants"] , "Stimulants")
#HD‐related mortality
add_mortality_data(t3["AAMR (HD)"],t2["All HD"] , "HD‐related mortality")
#SU+HD‐related mortality	
add_mortality_data(t5["AAMR (Overall)"],t1["Overall"] , "SU+HD‐related mortality")
#Sex
add_mortality_data(t5["AAMR (Male)"],t1["Men"] , "Male")
add_mortality_data(t5["AAMR (Female)"],t1["Women"] , "Female")
#Race
add_mortality_data(t6["AAMR (AI/AN)"],t1["AI/AN"] , "American Indian or Alaska Native")
add_mortality_data(t6["AAMR (A/PI)"],t1["A/PI"] , "Asian or Pacific Islander")
add_mortality_data(t6["AAMR (Black/AA)"],t1["Black/AA"] , "Black or African American")
add_mortality_data(t6["AAMR (White)"],t1["White"] , "White")
#Age
add_mortality_data(t7["Rate (Age 25-39 Years)"],t1["25-39 years"] , "25-39")
add_mortality_data(t7["Rate (Age 40-54 Years)"],t1["40-54 years"] , "40-54")
add_mortality_data(t7["Rate (Age 55-69 Years)"],t1["55-69 years"] , "55-69")
add_mortality_data(t7["Rate (Age 70-84 Years)"],t1["70-84 years"] , "70-84")
add_mortality_data(t7["Rate (Age 85+ Years)"],t1["85+ years"] , "85+")
#SUHD Drug Category
add_mortality_data(t8["Alcohol"],t2["Alcohol HD"] , "Alcohol + HD")
add_mortality_data(t8["Cannabis"],t2["Cannabis HD"] , "Cannabis + HD")
add_mortality_data(t8["Cocaine"],t2["Cocaine HD"] , "Cocaine + HD")
add_mortality_data(t8["Opioids"],t2["Opioids HD"] , "Opioid + HD")
add_mortality_data(t8["Sedatives/Hypnotics"],t2["Sedatives/Hypnotics HD"] , "Sedative-hypnotics + HD")
add_mortality_data(t8["Stimulants"],t2["Stimulants HD"] , "Stimulants + HD")

result.to_csv("T10_Presentation_Table_2005_2015.csv")
result.head(26)

,Category,Year 2005,95% CI (%),Year 2015,95% CI (%),Overall from 2005-2015,95% CI (%),AAPC (%),95% CI (%)
0,SU‐related mortality,36.75,36.48 to 37.02,51.49,51.19 to 51.79,42.57,42.49 to 42.66,None,None
1,Alcohol,21.20,21.00 to 21.40,27.69,27.48 to 27.91,23.64,23.58 to 23.70,None,None
2,Cannabis,0.10,0.08 to 0.11,0.32,0.30 to 0.35,0.17,0.17 to 0.18,None,None
3,Cocaine,4.58,4.48 to 4.67,4.21,4.12 to 4.30,3.83,3.80 to 3.85,None,None
4,Opioid,7.88,7.76 to 8.01,15.78,15.61 to 15.95,11.01,10.97 to 11.06,None,None
5,Sedative-hypnotics,2.26,2.19 to 2.33,5.36,5.26 to 5.46,3.78,3.75 to 3.81,None,None
6,Stimulants,1.13,1.09 to 1.18,3.75,3.66 to 3.83,1.77,1.75 to 1.78,None,None
7,HD‐related mortality,582.44,581.36 to 583.52,493.42,492.54 to 494.30,519.02,518.73 to 519.31,None,None
8,SU+HD‐related mortality,9.66,9.53 to 9.80,13.86,13.71 to 14.01,11.47,11.43 to 11.51,None,None
9,Male,15.77,15.51 to 16.02,21.55,21.28 to 21.83,18.14,18.06 to 18.22,None,None
